In [ ]:
# import findspark
# findspark.init()

from pyspark.sql import SparkSession

# Step 1: Initialize a SparkSession
spark = SparkSession.builder.master("local[*]") \
    .appName("LocalRowCountFoo") \
    .getOrCreate()


# # Step 2: Create an RDD with the "Hello World" message
# data = ["Hello", "World"]
# rdd = spark.sparkContext.parallelize(data)

# # Step 3: Perform an action to collect and print the data
# result = rdd.collect()
# print(" ".join(result))

# # Step 4: Stop the Spark session
# spark.stop()


In [ ]:
df = spark.read.csv('dummy_data.csv',header=True,inferSchema=True)
df.createOrReplaceTempView('customer')

df.printSchema()
df.show()


In [ ]:
sql_counts ='''
select 
    country, 
    count(*) as customer_count
from customer 
group by country
order by country
'''
sql_df = spark.sql(sql_counts)
sql_df.show()
sql_df.createOrReplaceTempView('customer_counts')

In [ ]:
sql_stats ='''

    SELECT 
        AVG(customer_count) AS avg_count, 
        STDDEV(customer_count) AS stddev_count,
        sum(customer_count) as total_customers
    FROM customer_counts

'''
avg_df = spark.sql(sql_stats)
avg_df.show()
avg_df.createOrReplaceTempView('customer_stats')

In [ ]:

sql_z ='''
SELECT 
    c.country,
    c.customer_count,
    (c.customer_count - s.avg_count) / s.stddev_count AS z_score,
    case when abs((c.customer_count - s.avg_count) / s.stddev_count) > 3 then 1 else 0 end   AS is_suspect
FROM customer_counts c
CROSS JOIN customer_stats s
order by c.country
'''
z_df = spark.sql(sql_z)
z_df.show()

In [ ]:
df_dauphine = spark.sql("select * from customer where firstname = 'John' and lastname='Dauphine'")

df_dauphine.show()
display(df_dauphine.count())
spark.stop()